# `black` formatter for the notebook

[`black`](https://github.com/ambv/black) is the newly popular, uncompromising formatter for Python.  When writing computational essays, the structure and format of code objects are important for readability.  

This notebook demonstrates the ability to format code cells with `black` interactively and with nbconvert.

In [116]:
    __all__ = 'Blacken', 'load_ipython_extension'

    %reload_ext deathbeds.__blacken__

The `%%blacken` magic will help interactively format code for sharing in a post or demostration.

* See the needlessly vulgar example below.

    %%blacken 100
    def some_long_ass_function(
                        with_="some"
            , fucked="up"     , *, parameters=None
    ):...

* becomes

    def some_long_ass_function(with_="some", fucked="up", *, parameters=None):
        ...


* and `ctrl+z` can be used to recover the original statement.

In [117]:
    from ast import literal_eval
    from IPython import get_ipython
    from textwrap import dedent, indent

    WIDTH = 80

In [118]:
    def blacken(line, str):
        ip = get_ipython()
        global WIDTH
        import black
        str = black.format_str(str, line and literal_eval(line) or WIDTH)
        if ip:
            ip.set_next_input(str, replace=True)
            return
        return str


* Create the IPython extension.

In [119]:
    def load_ipython_extension(ip):
        ip.register_magic_function(blacken, 'cell')

* Test the extension

In [120]:
    if __name__ == '__main__':
        load_ipython_extension(get_ipython())

## A `nbconvert` black preprocessor

In [121]:
    from nbconvert.preprocessors import Preprocessor
    from traitlets import Integer

In [122]:
    class Blacken(Preprocessor):
        width = Integer(default_value=WIDTH)
        def preprocess_cell(self, cell, resources=None, *args, **kwargs):
            if cell['cell_type'] == 'code':
                source = ''.join(cell['source'])
                indents = len(source) - len(source.lstrip())
                cell['source'] = indent(
                    blacken(
                        str(self.width), dedent(source)
                    ), ' '*indents
                )

            return cell, resources

## Use a configuration file for the nbconvert settings.

> I think we have to manually __import__ `importnb`.

Initialize the Blacken Preprocessor and append it to the available preprocessors.

    %%file config_blacken.py
    with __import__('importnb').Notebook():
        from deathbeds import __blacken__
    c.Exporter.preprocessors.append(__blacken__.Blacken())

## `nbconvert` usage

In [123]:
    if __name__ == '__main__':
        get_ipython().system("""
        
        
        jupyter nbconvert --config config_blacken.py --to notebook 2018-07-04-blacken-the-interactive-black-formatter.ipynb
        
        
        
        """.strip())

[NbConvertApp] Converting notebook 2018-07-04-blacken-the-interactive-black-formatter.ipynb to notebook
[NbConvertApp] Writing 6634 bytes to 2018-07-04-blacken-the-interactive-black-formatter.nbconvert.ipynb


In [124]:
    if __name__ == '__main__': import disqus